##### Compute SOFA score

Connect to database

In [28]:
import __init__

In [29]:
import numpy as np
import pandas as pd
import datetime

# Open a connection to OMOP
from fleming_lib.tools import connect_to_omop
from fleming_lib.severity_scores import compute_sofa_score
conn = connect_to_omop()

In [30]:
patient_id = (62063368, 62106569)  # person_id

In [56]:
# Get data

query = """
select
    distinct m.person_id, m.measurement_datetime,m.measurement_concept_id, m.measurement_concept_name, m.value_source_value, m.unit_source_value
from 
    measurement m 

where
    m.measurement_concept_id in
    (3024929,   -- platelets
     3024128,   -- bilirubin
     3027598,   -- map_bp
     3016723 ,  -- creatinine
     3032652    -- glasgow coma scale
    )
  
    
and m.person_id in {}
order by m.person_id, m.measurement_datetime
;""".format(patient_id)

measures = pd.read_sql_query(query, conn)
measures.iloc[:10]

DatabaseError: Execution failed on sql: 
select
    distinct m.person_id, m.measurement_datetime,m.measurement_concept_id, m.measurement_concept_name, m.value_source_value, m.unit_source_value
from 
    measurement m 

where
    m.measurement_concept_id in
    (3024929,   -- platelets
     3024128,   -- bilirubin
     3027598,   -- map_bp
     3016723 ,  -- creatinine
     3032652    -- glasgow coma scale
    )
  
    
and m.person_id in (62063368, 62106569)
order by m.person_id, m.measurement_datetime
;
[WinError 10060] Une tentative de connexion a échoué car le parti connecté n’a pas répondu convenablement au-delà d’une certaine durée ou une connexion établie a échoué car l’hôte de connexion n’a pas répondu
unable to rollback

#### Change type datetime

In [33]:
# Select date range
df=measures
df = df.set_index(['measurement_datetime'])
df=df.loc['2188-11-12 13:00:00':'2188-11-13 13:00:00']

In [54]:
#get features

#pa02
pa=0

# Platelets
platelets=df[df['measurement_concept_name']=='Platelets [#/volume] in Blood by Automated count']
pla=platelets['value_source_value']
pla=pla[0]
pla=pd.to_numeric(pla)

# Total Bilirubin serum/plasma
bilirubin=df[df['measurement_concept_name']=='Total Bilirubin serum/plasma']
bil=bilirubin['value_source_value']
bil=bil[0]
bil=pd.to_numeric(bil)

# Mean blood pressure
mean_blood_pressure=df[df['measurement_concept_name']=='Mean blood pressure']
mbp=mean_blood_pressure['value_source_value']
mbp=mbp[0]
mbp=pd.to_numeric(mbp)

# Dopamine
dop=0

# Glasgow coma scale
glasgow_coma_scale=df[df['measurement_concept_name']=='Glasgow coma scale']
gcs=glasgow_coma_scale['value_source_value']
gcs=gcs[0]
gcs=pd.to_numeric(gcs)

# Creatinine
creatinine=df[df['measurement_concept_name']=='Creatinine serum/plasma']
cre=creatinine['value_source_value']
cre=cre[0]
cre=pd.to_numeric(cre)


In [55]:
# Compute SOFA score
score=compute_sofa_score(pa,pla,bil,mbp,dop,gcs,cre)
print(score)

10
